# **1-1. 통합 테스트**

## **1.환경설정**

### (1) 경로 설정

로컬 경로 설정

In [13]:
path = './'

### (2) 라이브러리

In [51]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 더 필요한 라이브러리 추가 -------------
import my_emergency as em # 강사가 만든 모듈. 나중에 내가 만든 모듈 불러서 해야한다.

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [52]:
em.load_key(path)

### (2) audio to text

In [53]:
audio_path = path + 'audio/'
result = em.audio2text(audio_path)
result

,Audio,Transcribed
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
1,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."
4,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...


### (3) text summary

In [54]:
result = em.text2summary(path, result)
result

,Audio,Transcribed,summary
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,"{""summary"": ""머리에서 출혈 후 어지러움이 있음"", ""부위"": ""머리"", ..."
1,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,"{""summary"": ""38도 열이 나면서 머리가 아프고 띵한 증상이 있음"", ""부..."
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,"{""summary"": ""코에 콩이 들어가 막혀있음"", ""부위"": ""코"", ""증상"":..."
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...","{""summary"": ""머리를 박았는데 3시간 후에 머리가 어지럽고 속이 메스꺼움""..."
4,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,"{""summary"": ""미끄러져 엉덩방아를 찍고 꼬리뼈가 계속 아프다"", ""부위"":..."


### (4) 응급실 등급분류

In [55]:
predicted_class, probabilities = em.model_prediction(path,result)

print(f"예측된 클래스: {predicted_class}")
print(f"예측된 클래스 이름: {predicted_class+1}등급")
print(f"클래스별 확률: {probabilities}")

예측된 클래스: 2
예측된 클래스 이름: 3등급
클래스별 확률: tensor([[0.0146, 0.1916, 0.5491, 0.1864, 0.0584]])


### (5) 응급실추천

In [56]:
df = pd.read_excel(path+'/audio_location.xlsx')
# em.get_dist(path, df)
em.recom_em(path, df)

탐색범위:11.0km * 11.0km
(2.626, '분당서울대학교병원', '031-787-2114', '031-787-3119')
(4.789, '대진의료재단분당제생병원', '031-779-0114', '031-779-0119')
(5.973, '국군수도병원', '031-725-6075', '031-725-6119')


## **3. 통합 테스트**

* 세부사항
    * [2. 단위 테스트] 의 내용을 순차적으로 실행하며 결과를 확인합니다.

In [57]:
# 1. 준비 작업 ---------------------------------------------
em.load_key(path)
result = em.audio2text(path + 'audio/')
result = em.text2summary(path, result)
predicted_class, probabilities = em.model_prediction(path,result)

print(f"예측된 클래스: {predicted_class}")
print(f"예측된 클래스 이름: {predicted_class+1}등급")
print(f"클래스별 확률: {probabilities}")

if predicted_class <= 2 :
    df = pd.read_excel(path+'/audio_location.xlsx')
    em.recom_em(path, df)
else :
    print('개인 건강관리')

# # 테스트 input --------------------
# predicted_class = 2  # 테스트용
# start_lat, start_lng = 35.548238, 129.307011
# # ---------------------------------

# if predicted_class <= 2 :
#     result = em.recommend_hospital3(path, emergency, start_lat, start_lng, c_id, c_key)
#     display(result)
# else :
#     print('개인 건강관리')

예측된 클래스: 1
예측된 클래스 이름: 2등급
클래스별 확률: tensor([[0.0249, 0.4331, 0.4059, 0.0898, 0.0463]])
탐색범위:11.0km * 11.0km
(2.626, '분당서울대학교병원', '031-787-2114', '031-787-3119')
(4.789, '대진의료재단분당제생병원', '031-779-0114', '031-779-0119')
(5.973, '국군수도병원', '031-725-6075', '031-725-6119')
